In [2]:
import pandas as pd 
import numpy as np
import pyodbc
import seaborn as sns 
import matplotlib.pyplot as plt 
import matplotlib as mpl
from copy import copy

from IPython.display import set_matplotlib_formats

# 데이터 전처리
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import os

import tensorflow as tf
from tensorflow import keras

# keras.io 예제 코드에서는 from keras.models import Sequential로 되어있지만, 앞에 tensorflow 넣어주어야 함
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [16]:
%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease [4,003 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 Packages [184 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key output should not be parsed (stdout is not a terminal)
100   983  100   983    0     0  15854      0 --:--:-- --:--:-- --:--:-- 15854
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    79  100    79    0     0   3291      0 --:--:-- --:--:-- --:--:--  3291
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling

In [17]:
server = 'daesangpoc.koreacentral.cloudapp.azure.com'  # 서버 주소 (DB 접속 ip)
database = 'daesang'  # 데이터베이스명
username = 'sa'  # 접속 유저
password = 'p@ssw0rd'   # 패스워드
driver = '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+ ';SERVER='+server+ ';DATABASE='+database+ ';UID='+username+ ';PWD='+ password)
cursor = cnxn.cursor()

In [18]:
SQL_query = pd.read_sql_query(
                              '''
                                SELECT * 
                                 FROM FST_ANAL_MART
                                ORDER BY ITEM_CODE, SALE_DATE
                              ''', cnxn)
df = pd.DataFrame(SQL_query)

In [20]:
week_df = df[['YYYYWW','SALE_DATE','WEEK_ORDER','YYYY','MM','WW']].drop_duplicates()

In [21]:
# series_to_supervised()  series 데이터를 지도 학습 문제로 변환하는 것
# 시계열 데이터는 input(x)과 output(y)의 시퀀스 (지도 학습)으로 변환되어야 한다.
# time series forecasting problem to supervised learning problem

# n_in  ==> t-1 (lagged observations), X 데이터 만드는 것
# n_out ==> t  (current time step), y 데이터 만드는 것

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X). 
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [22]:
# 과거 1년 (52주) (변경 가능) 데이터로 향후 4개구간 (한달)을 예측할 것
# n_in = 52 (t-1, t-2, ..., t-52)
# n_out = 4 (t, t+1, t+2, t+3)

# train / test 나누기
# transform series into train and test sets for supervised learning

# n_lag = n_in (52) 과거 학습 구간
# n_seq = n_out (4) 미래 예측 구간

# transform series into train and test sets for supervised learning
def prepare_data(series, n_lag, n_seq):
    # extract raw values
    raw_values = series.values
    raw_values = raw_values.reshape(len(raw_values), 1)
    # rescale values to 0, 1
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_values = scaler.fit_transform(raw_values)
    scaled_values = scaled_values.reshape(len(scaled_values), 1)
    # transform into supervised learning problem X, y
    supervised = series_to_supervised(scaled_values, n_lag, n_seq)
    supervised_values = supervised.values
    # split into train and test sets   # 전체 데이터의 80%를 학습 데이터로 사용
    n_test = int(round(supervised.shape[0] * 0.2))
    train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
    return scaler, train, test

In [23]:
# 모델 선언 & 컴파일
model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20),
    keras.layers.Dense(4)
])

optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss="mean_squared_error", optimizer=optimizer)

In [24]:
dmd_cnt = 12

In [ ]:
result_dict = dict()
for plan_week_id in ['W202015','W202016','W202017','W202018','W202019','W202020','W202021','W202022'] :
  plan_week_order = week_df[week_df['YYYYWW']==plan_week_id]['WEEK_ORDER'].iloc[0]
  fcst_start_date = week_df[week_df['YYYYWW']==plan_week_id]['SALE_DATE'].iloc[0]
  fcst_week_order = week_df[(week_df['WEEK_ORDER']>=plan_week_order)&(week_df['WEEK_ORDER']<=plan_week_order+3)]['WEEK_ORDER'].to_list()
  fcst_date_seq = week_df[week_df['WEEK_ORDER'].isin(fcst_week_order)]['SALE_DATE'].to_list()

  # 기존 컬럼명 저장
  cols_list = df.columns

  # 새로운 데이터프레임 생성
  df_target = copy(df)

  # 수요 실적(SO_BOX) 준비
  df_dmd = df_target[['ITEM_CODE','SALE_DATE','SO_BOX']]  # 원본 수요실적 저장
  df_dmd.rename({'SALE_DATE':'FCST_SALE_DATE'}, axis='columns', inplace=True)
  df_dmd['FCST_SALE_DATE'] = pd.to_datetime(df_dmd['FCST_SALE_DATE'])

  df_target['SO_BOX_MA'] = df_target.groupby(by='ITEM_CODE')['SO_BOX'].rolling(5).mean().reset_index(drop=True)
  df_target['SO_BOX_FIN'] = np.where(df_target['SALE_DATE'].isin(fcst_date_seq), df_target['SO_BOX_MA'], df_target['SO_BOX'])   # 예측 구간에 해당하는 일자에만 MA 값을 넣음

  df_target = df_target.drop(columns=['SO_BOX','SO_BOX_MA','SO_LAG_W1','SO_LAG_W2','SO_LAG_W3','SO_LAG_M','SO_LAG_Y'], inplace=False)  # 불필요한 컬럼 삭제
  df_target = df_target.rename({'SO_BOX_FIN':'SO_BOX'}, axis='columns', inplace=False)

  # 수요의 LAG값 구하기
  df_target['SO_LAG_W1'] = df_target.groupby(by='ITEM_CODE')['SO_BOX'].shift(1)
  df_target['SO_LAG_W2'] = df_target.groupby(by='ITEM_CODE')['SO_BOX'].shift(2)
  df_target['SO_LAG_W3'] = df_target.groupby(by='ITEM_CODE')['SO_BOX'].shift(3)
  df_target['SO_LAG_M'] = df_target.groupby(by='ITEM_CODE')['SO_BOX'].shift(4)
  df_target['SO_LAG_Y'] = df_target.groupby(by='ITEM_CODE')['SO_BOX'].shift(52)

  # 컬럼을 기존 데이터프레임처럼 정렬
  df_target = df_target[cols_list]

  # SALE_DATE, SALE_START_DATE 날짜 타입으로 변환
  df_target['SALE_DATE'] = pd.to_datetime(df_target['SALE_DATE'])
  df_target['SALE_START_DATE'] = pd.to_datetime(df_target['SALE_START_DATE'])

  # PLAN_WEEK_ID에 해당하는 수요 구간 필터링
  df_target = df_target[df_target['WEEK_ORDER']<=max(fcst_week_order)]

  # YYYY, MM, WW 숫자(int) 형식으로 변환
  df_target = df_target.astype({'YYYY':int, 'MM':int, 'WW':int})

  # LAG 변수에 있는 NULL값은 0으로 대체
  df_target = df_target.fillna(0, inplace=False)

  # 모델에 필요한 변수만 남기기
  drop_cols = ['LOCATION','PLAN_BOX','YYYYWW']
  df_target = df_target.drop(columns = drop_cols, inplace=False)

  ts_df = df_target.pivot_table(index='ITEM_CODE', values='SO_BOX', columns='WEEK_ORDER')
  ts_df.columns = ts_df.columns.astype(str)
  ts_df['CNT'] = ts_df.count(axis=1)   # 수요 개수(구간) 컬럼 추가
  ts_df = ts_df[ts_df['CNT']>=208]  # 수요 개수(구간) 208개 이상 (4년 이상)
  item_code = ts_df.reset_index(inplace=False)['ITEM_CODE']

  forecast_dict = dict()
  for index, row in ts_df.iterrows() :
    n_lag = dmd_cnt
    n_seq = 4
    features = df_target[df_target['ITEM_CODE']==index][['SO_BOX']]
    features.index = df_target[df_target['ITEM_CODE']==index]['SALE_DATE']
    scaler, train, test = prepare_data(features, n_lag, n_seq)

    n_valid = int(round(train.shape[0]) * 0.2)
    train, valid = train[:-n_valid, :], train[:n_valid, :]
    train = train.reshape(train.shape[0], train.shape[1], 1)
    valid = valid.reshape(valid.shape[0], valid.shape[1], 1)
    test = test.reshape(test.shape[0], test.shape[1], 1)

    X_train, Y_train = train[:, :n_lag], train[:, -n_seq:, 0]
    X_valid, Y_valid = valid[:, :n_lag], valid[:, -n_seq:, 0]
    X_test, Y_test = test[:, :n_lag], test[:, -n_seq:, 0]

    history = model.fit(X_train, Y_train, epochs=300, shuffle=False, validation_data=(X_valid, Y_valid))
    Y_pred = model.predict(X_test)
    Y_pred_inv = scaler.inverse_transform(Y_pred)
    Y_test_inv = scaler.inverse_transform(Y_test)
    Y_pred_fin = Y_pred_inv[-1]
    Y_test_fin = Y_test_inv[-1]

    forecast_dict[index] = Y_pred_fin
  
  df_result = pd.DataFrame(forecast_dict)
  df_result.index = fcst_date_seq
  df_result = pd.melt(df_result, ignore_index=False)
  df_result.reset_index(inplace=True)
  df_result.columns = ['SALE_DATE', 'ITEM_CODE', 'FCST_BOX']

  df_result['PLAN_WEEK_ID'] = plan_week_id
  df_result['FCST_START_DATE'] = fcst_start_date
  df_result['SCN_ID'] = 'SCN001'
  df_result['LOCATION'] = 'HQ'
  df_result['METHOD'] = 'DNN_GRU'
  df_result = pd.merge(df_result, week_df[['SALE_DATE','YYYYWW','WEEK_ORDER']], how='left', on = ['SALE_DATE'])
  df_result.rename({'SALE_DATE':'FCST_SALE_DATE', 'YYYYWW':'FCST_YYYYWW', 'WEEK_ORDER':'FCST_WEEK_ORDER'}
                  , axis='columns', inplace=True)
  df_result['FCST_SALE_DATE'] = pd.to_datetime(df_result['FCST_SALE_DATE'])
  df_result['FCST_START_DATE'] = pd.to_datetime(df_result['FCST_START_DATE'])
  df_result = pd.merge(df_result, df_dmd, how='left', on=['ITEM_CODE','FCST_SALE_DATE'])
  df_result = df_result[['PLAN_WEEK_ID','FCST_START_DATE','SCN_ID','ITEM_CODE','LOCATION','METHOD','FCST_SALE_DATE'
                        ,'FCST_YYYYWW', 'FCST_WEEK_ORDER', 'SO_BOX', 'FCST_BOX']]
  df_result['ERR'] = df_result['FCST_BOX'] - df_result['SO_BOX']

  result_dict[plan_week_id] = df_result

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
6/6 [==============================] - 0s 6ms/step - loss: 0.0380 - val_loss: 0.0456
Epoch 132/300
6/6 [==============================] - 0s 6ms/step - loss: 0.0379 - val_loss: 0.0456
Epoch 133/300
6/6 [==============================] - 0s 6ms/step - loss: 0.0379 - val_loss: 0.0455
Epoch 134/300
6/6 [==============================] - 0s 6ms/step - loss: 0.0379 - val_loss: 0.0454
Epoch 135/300
6/6 [==============================] - 0s 6ms/step - loss: 0.0379 - val_loss: 0.0454
Epoch 136/300
6/6 [==============================] - 0s 6ms/step - loss: 0.0378 - val_loss: 0.0453
Epoch 137/300
6/6 [==============================] - 0s 7ms/step - loss: 0.0378 - val_loss: 0.0453
Epoch 138/300
6/6 [==============================] - 0s 7ms/step - loss: 0.0378 - val_loss: 0.0452
Epoch 139/300
6/6 [==============================] - 0s 7ms/step - loss: 0.0378 - val_loss: 0.0452
Epoch 140/300
6/6 [==============================] - 0s 7ms/step - loss: 0.0377 - val_l

In [ ]:
dnn_result = pd.DataFrame(columns = df_result.columns)
for i in result_dict.keys():
  dnn_result = dnn_result.append(result_dict[i])

In [ ]:
server = 'daesangpoc.koreacentral.cloudapp.azure.com'  # 서버 주소 (DB 접속 ip)
database = 'daesang'  # 데이터베이스명
username = 'sa'  # 접속 유저
password = 'p@ssw0rd'   # 패스워드
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

driver = '{ODBC Driver 17 for SQL Server}'
params = 'DRIVER='+driver + ';SERVER='+server + ';DATABASE='+database + ';UID='+username + ';PWD='+ password
db_params = urllib.parse.quote_plus(params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    print("FUNC call")
    if executemany:
        cursor.fast_executemany = True

table_name = 'FST_RESULT_DNN'
dnn_result.to_sql(table_name, engine , index=False, if_exists="append", schema="dbo")